### convert the PTF data MJD to BJD to match the ZTF data. 

Currently, our PTF data is in MJD and looks like "56078.42698", and our ZTF data is in BJD (TDB) and looks like "2458268.94714".

In [1]:
MJD = 56078.42698
JD = MJD+2400000.5
print(JD)

2456078.92698


Kupfer game me the following code to use.

In [2]:
from astropy import time, coordinates as coord, units as u 

#a singular MJD for "J003950.86+405332.0" along with its RA and DEC in deg.:
#mjd = 56078.42698
RA = 9.96183
DEC = 40.89221

def MJD2BJD(MJD,ra,dec,verbose=False):
    target = coord.SkyCoord(ra,dec,unit=(u.deg, u.deg), frame='icrs')
    
    #observatory = coord.EarthLocation.of_site('Palomar')
    observatory = coord.EarthLocation.from_geodetic(243.137,33.356,1706)  #Palomar
    #observatory = coord.EarthLocation.from_geodetic(204.5253,19.8260,4145) #Keck
    #observatory = coord.EarthLocation.from_geodetic(342.118,28.761,2332) #WHT
    #observatory = coord.EarthLocation.from_geodetic(203.7432,20.7082,3052) #Haleakala (OGG)
    #observatory = coord.EarthLocation.from_geodetic(149.0685,-31.2749,1165) #Siding Springs (COJ)
  
    times = time.Time(MJD+0.0001736111, format='mjd', scale='utc', location=observatory)
    ltt_bary = times.light_travel_time(target)
    time_barycentre = times.tdb + ltt_bary
    return time_barycentre.mjd
#this is actually mbjd
bjd = MJD2BJD(mjd,RA,DEC,verbose=False)
#NOW this is bjd
bjd+2400000.5 

2456078.9243315333

In [111]:
#trying to accomodate to a list of MJDs:

import numpy as np

data = np.loadtxt(open('/Volumes/MeganSSD/KupferResearch/Data/old_data/J003950.86+405332.0.csv'),
                  delimiter=",", skiprows=1)

MJD = data[:,0]
print('MJD:',MJD)
print('length of MJD:',len(MJD))

RA = 9.96183
DEC = 40.89221

def MJD2BJD(MJD,ra,dec,verbose=False):
    target = coord.SkyCoord(ra,dec,unit=(u.deg, u.deg), frame='icrs')
    
    #observatory = coord.EarthLocation.of_site('Palomar')
    observatory = coord.EarthLocation.from_geodetic(243.137,33.356,1706)  #Palomar
    #observatory = coord.EarthLocation.from_geodetic(204.5253,19.8260,4145) #Keck
    #observatory = coord.EarthLocation.from_geodetic(342.118,28.761,2332) #WHT
    #observatory = coord.EarthLocation.from_geodetic(203.7432,20.7082,3052) #Haleakala (OGG)
    #observatory = coord.EarthLocation.from_geodetic(149.0685,-31.2749,1165) #Siding Springs (COJ)

    times = time.Time(MJD+0.0001736111, format='mjd', scale='utc', location=observatory)
    #0.0001736111 is 15 seconds since BJD is a 'mid-time' exposure
    ltt_bary = times.light_travel_time(target)
    time_barycentre = times.tdb + ltt_bary
    
    return time_barycentre.mjd

bjd = MJD2BJD(MJD,RA,DEC,verbose=False)

#goes through the array of MJD times per target, and converts them to BJD
newjd = bjd+2400000.5
print('newjd:',newjd)
print('length of newjd:',len(newjd))
    
#np.savetxt('/Volumes/MeganSSD/KupferResearch/Data/old_data/newbjd/outputbjd.txt', newjd, header ='BJD', 
#           delimiter=',', fmt='%1.5f')

MJD: [56078.42698 56078.45757 56079.42315 ... 57761.12808 57798.12679
 57798.14851]
length of MJD: 1643
newjd: [2456078.92433153 2456078.95492326 2456079.92055729 ... 2457761.6298422
 2457798.62552567 2457798.64724403]
length of newjd: 1643


In [89]:
data = np.loadtxt(open('/Volumes/MeganSSD/KupferResearch/Data/old_data/J003950.86+405332.0.csv'),
                  delimiter=",", skiprows=1)
print(data)



[[5.60784270e+04 1.87621982e+01 1.01258426e-01]
 [5.60784576e+04 1.89167413e+01 1.09500645e-01]
 [5.60794232e+04 1.88100861e+01 1.04395794e-01]
 ...
 [5.77611281e+04 1.86092005e+01 1.36899699e-01]
 [5.77981268e+04 1.85907723e+01 1.19446191e-01]
 [5.77981485e+04 1.87574566e+01 2.27595593e-01]]


In [105]:
newjd = np.reshape(newjd, (data.shape[0],1))   # adjust dimension of the new array
result = np.append(data, newjd, 1)  

In [107]:
np.savetxt('/Volumes/MeganSSD/KupferResearch/Data/old_data/J003950.86+405332.0.csv', result, delimiter=" ", fmt="%s")